Fuente: https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

In [1]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

In [2]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [3]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [4]:
# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation


In [5]:
# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))



In [6]:
# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs


In [7]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

In [20]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
					print("error: {}".format(error))
				errors.append(error)
		else:
			("In last layer...")
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
				print("error: {}".format(neuron['output'] - expected[j]))
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [37]:
# Update network weights with error
def update_weights(network, row, l_rate):
	print("Updating weights")
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

In [38]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

In [39]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

In [40]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))


In [41]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)


In [42]:
# load and prepare data
filename = '../data/wheat-seeds.csv'
dataset = load_csv(filename)
dataset

[['15.26', '14.84', '0.871', '5.763', '3.312', '2.221', '5.22', '1'],
 ['14.88', '14.57', '0.8811', '5.554', '3.333', '1.018', '4.956', '1'],
 ['14.29', '14.09', '0.905', '5.291', '3.337', '2.699', '4.825', '1'],
 ['13.84', '13.94', '0.8955', '5.324', '3.379', '2.259', '4.805', '1'],
 ['16.14', '14.99', '0.9034', '5.658', '3.562', '1.355', '5.175', '1'],
 ['14.38', '14.21', '0.8951', '5.386', '3.312', '2.462', '4.956', '1'],
 ['14.69', '14.49', '0.8799', '5.563', '3.259', '3.586', '5.219', '1'],
 ['14.11', '14.1', '0.8911', '5.42', '3.302', '2.70', '5.00', '1'],
 ['16.63', '15.46', '0.8747', '6.053', '3.465', '2.04', '5.877', '1'],
 ['16.44', '15.25', '0.888', '5.884', '3.505', '1.969', '5.533', '1'],
 ['15.26', '14.85', '0.8696', '5.714', '3.242', '4.543', '5.314', '1'],
 ['14.03', '14.16', '0.8796', '5.438', '3.201', '1.717', '5.001', '1'],
 ['13.89', '14.02', '0.888', '5.439', '3.199', '3.986', '4.738', '1'],
 ['13.78', '14.06', '0.8759', '5.479', '3.156', '3.136', '4.872', '1'],
 [

In [43]:
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
dataset

[[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, '1'],
 [14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, '1'],
 [14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825, '1'],
 [13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805, '1'],
 [16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175, '1'],
 [14.38, 14.21, 0.8951, 5.386, 3.312, 2.462, 4.956, '1'],
 [14.69, 14.49, 0.8799, 5.563, 3.259, 3.586, 5.219, '1'],
 [14.11, 14.1, 0.8911, 5.42, 3.302, 2.7, 5.0, '1'],
 [16.63, 15.46, 0.8747, 6.053, 3.465, 2.04, 5.877, '1'],
 [16.44, 15.25, 0.888, 5.884, 3.505, 1.969, 5.533, '1'],
 [15.26, 14.85, 0.8696, 5.714, 3.242, 4.543, 5.314, '1'],
 [14.03, 14.16, 0.8796, 5.438, 3.201, 1.717, 5.001, '1'],
 [13.89, 14.02, 0.888, 5.439, 3.199, 3.986, 4.738, '1'],
 [13.78, 14.06, 0.8759, 5.479, 3.156, 3.136, 4.872, '1'],
 [13.74, 14.05, 0.8744, 5.482, 3.114, 2.932, 4.825, '1'],
 [14.59, 14.28, 0.8993, 5.351, 3.333, 4.185, 4.781, '1'],
 [13.99, 13.83, 0.9183, 5.119, 3.383, 5.234, 4.781, '1'],
 [15.69, 14.75, 0.9058, 5.

In [44]:
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
dataset

[[15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22, 2],
 [14.88, 14.57, 0.8811, 5.554, 3.333, 1.018, 4.956, 2],
 [14.29, 14.09, 0.905, 5.291, 3.337, 2.699, 4.825, 2],
 [13.84, 13.94, 0.8955, 5.324, 3.379, 2.259, 4.805, 2],
 [16.14, 14.99, 0.9034, 5.658, 3.562, 1.355, 5.175, 2],
 [14.38, 14.21, 0.8951, 5.386, 3.312, 2.462, 4.956, 2],
 [14.69, 14.49, 0.8799, 5.563, 3.259, 3.586, 5.219, 2],
 [14.11, 14.1, 0.8911, 5.42, 3.302, 2.7, 5.0, 2],
 [16.63, 15.46, 0.8747, 6.053, 3.465, 2.04, 5.877, 2],
 [16.44, 15.25, 0.888, 5.884, 3.505, 1.969, 5.533, 2],
 [15.26, 14.85, 0.8696, 5.714, 3.242, 4.543, 5.314, 2],
 [14.03, 14.16, 0.8796, 5.438, 3.201, 1.717, 5.001, 2],
 [13.89, 14.02, 0.888, 5.439, 3.199, 3.986, 4.738, 2],
 [13.78, 14.06, 0.8759, 5.479, 3.156, 3.136, 4.872, 2],
 [13.74, 14.05, 0.8744, 5.482, 3.114, 2.932, 4.825, 2],
 [14.59, 14.28, 0.8993, 5.351, 3.333, 4.185, 4.781, 2],
 [13.99, 13.83, 0.9183, 5.119, 3.383, 5.234, 4.781, 2],
 [15.69, 14.75, 0.9058, 5.527, 3.514, 1.599, 5.046, 2],
 [14

In [45]:
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
dataset

[[0.4409820585457979,
  0.5020661157024793,
  0.570780399274047,
  0.48648648648648646,
  0.48610121168923714,
  0.18930164220052273,
  0.3451501723289019,
  2],
 [0.40509915014164316,
  0.44628099173553726,
  0.6624319419237747,
  0.3688063063063065,
  0.5010691375623664,
  0.03288301759221938,
  0.21516494337764666,
  2],
 [0.3493862134088762,
  0.3471074380165289,
  0.8793103448275864,
  0.22072072072072094,
  0.50392017106201,
  0.25145301590191005,
  0.15066469719350079,
  2],
 [0.3068932955618508,
  0.31611570247933873,
  0.7931034482758617,
  0.23930180180180172,
  0.5338560228082679,
  0.19424254638598865,
  0.14081733136386,
  2],
 [0.5240793201133145,
  0.5330578512396694,
  0.8647912885662429,
  0.4273648648648651,
  0.6642908054169634,
  0.076701036289641,
  0.32299359921221066,
  2],
 [0.3578847969782815,
  0.37190082644628114,
  0.7894736842105262,
  0.2742117117117118,
  0.48610121168923714,
  0.22063737663992516,
  0.21516494337764666,
  2],
 [0.3871576959395656,
  0.42

In [ ]:
# Test Backprop on Seeds dataset
seed(0)
# evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

error: -0.08056434626870479
error: 0.9657204821155605
error: 0.9478824266834511
error: -0.005418911112559891
error: -0.00382500145048227
error: 0.04199823966597837
error: -0.003604356959381178
error: 0.022697693387333424
error: 0.06191259984254448
error: -4.1274341226437986e-07
error: 0.03114366032649909
error: 0.03725555728288883
error: -0.00023220437615073118
error: 0.013039584469113205
error: 0.01372930768803164
error: -0.0019451360334269343
error: 0.015592189414096326
error: 0.06005632357397853
Updating weights
error: 0.9321859232262473
error: -0.026982602953497636
error: 0.9571526723895722
error: 0.05359704845223326
error: 0.053567375492954435
error: 0.09152214609518325
error: 0.035683867827171964
error: 0.03510699213875772
error: 0.0674972709266272
error: 9.00748959816977e-05
error: -0.0005945090298732016
error: 0.00407950584040059
error: 0.002381009689144829
error: 0.00209259306467684
error: 0.002210321391001707
error: 0.019298210655401167
error: 0.01891545111750022
error: 0.055

error: -0.032685404124190416
error: -0.04697578398705267
Updating weights
error: -0.5893152056312413
error: 0.3698948196232786
error: 0.22850024395554455
error: -0.04639412829641353
error: -0.09976297436583623
error: -0.0902507941875583
error: -0.007879164782974828
error: -1.6971328300339344e-05
error: 0.0014933306851136212
error: 0.09672199416551347
error: 0.12998634404217793
error: 0.10530118722836013
error: 0.06770958498999827
error: 0.0405486740162143
error: 0.009019423153007695
error: 0.02998206241908413
error: 0.015074033995205408
error: 0.021101912489146724
Updating weights
error: 0.44499481157294424
error: 0.33064947055252303
error: -0.7909628655299431
error: 0.03952166781254817
error: -0.0072976712950007655
error: -0.0369119198835839
error: 0.009998198719054155
error: 0.015091346084616297
error: 0.01150774372059405
error: -0.07092264068135225
error: -0.04413863197564052
error: 0.03721594917070179
error: -0.0483241297905287
error: -0.0729285354199594
error: 0.03072759222925983


error: -0.06844787565030544
error: 0.001112631575796301
error: 0.016777917539511463
error: 0.013256099918030652
error: -0.08174178488237858
error: -0.010098425409777387
error: 0.06853443005108738
error: -0.04028448055053925
error: -0.06741552471597367
error: 0.04201007412577441
error: -0.020312817005037183
error: -0.03157190645960377
error: -0.049425853430407035
Updating weights
error: 0.3449956900551245
error: 0.3763738493041866
error: -0.7353024884850483
error: 0.018762044706885955
error: -0.03278451680639881
error: -0.07317880034673063
error: -0.0006672519230869022
error: 0.010469451070245874
error: 0.0015636236807986831
error: -0.07877797267818033
error: -0.018836841879105053
error: 0.06013346919730737
error: -0.039790751094389724
error: -0.06562207311733137
error: 0.045284073427335314
error: -0.020922619170430968
error: -0.0330588963712312
error: -0.05711339905216023
Updating weights
error: -0.6700190873323407
error: 0.352413105815592
error: 0.30782475907061346
error: -0.032599906

error: 0.005577536954150684
error: 0.013303343443087968
error: 0.0213143317536866
error: 0.2075210337565543
error: 0.28128076859655327
error: 0.24306969864788738
error: 0.07222326827164437
error: 0.051922891943609606
error: -0.008458974217762115
error: 0.04182409716628905
error: 0.03225053291837272
error: 0.04849824367647042
Updating weights
error: -0.578260543726502
error: 0.2997735756761889
error: 0.31944278953614164
error: -0.03578494493652394
error: -0.0761721921304996
error: -0.053813847860780674
error: 0.0007931137899343573
error: 0.007252515151971801
error: 0.012916332067521404
error: 0.20470739087368847
error: 0.27530657938186154
error: 0.24138256774563988
error: 0.06724005077184794
error: 0.046704649121484754
error: -0.007268562309387314
error: 0.03691991612394886
error: 0.026714097282839223
error: 0.03955787116540554
Updating weights
error: 0.44079053156224335
error: 0.29588852628396667
error: -0.7038495917384564
error: 0.031591969798706564
error: -0.00899155160121174
error: 

error: -0.055213753525272455
error: -0.2821109832209742
error: -0.3108414166986776
error: -0.013381276127918978
error: 0.02099841827837525
error: -0.031248172547023265
error: -0.008758634391239266
error: 0.004692901463815322
error: 0.018732595008208654
Updating weights
error: -0.5929454685081225
error: 0.35027211436458006
error: 0.3472520667392602
error: -0.02930041191421644
error: -0.07856746836602069
error: -0.05132181986860833
error: 0.012571140543984548
error: 0.02787303147368553
error: 0.03465341600631421
error: 0.3035530944348587
error: 0.45501491778367587
error: 0.419858364081249
error: 0.07437503722491287
error: 0.05444577537541194
error: -0.008600216978703681
error: 0.0490596148740618
error: 0.042940939936238355
error: 0.058015423944555285
Updating weights
error: 0.27104265870713795
error: 0.46457867180582063
error: -0.7093626289948556
error: 0.012857147148258473
error: -0.06083974927732834
error: -0.10861953414279926
error: -0.0029160830141428356
error: 0.01711765694454794
er

...

error: 0.0004085266317260209
error: -0.12792441323874354
error: -0.26233458913017954
error: -0.0012997977374010067
error: -0.014563253913370133
error: -0.08091642310679874
error: -0.015169393868617282
error: 0.4503998836519862
error: 0.5656990764473964
error: 0.003530499572731914
error: 0.06078654020319543
error: 0.680020498610037
error: -0.0036724884561181227
error: -0.05177441247351718
error: -0.2798705669862525
Updating weights
error: -0.1192842615232036
error: 0.03258813294789293
error: 0.11116367822805719
error: -0.002278322563130263
error: -0.004120440119411975
error: 0.006626434349119907
error: 0.007282059186261231
error: 0.007074269797686647
error: 0.012570520691328933
error: 0.0849041476198198
error: 0.09149670969657786
error: 0.08292997940403256
error: -0.019754558355665523
error: -0.018955027029374627
error: -0.0661000402137969
error: 0.02056083404876791
error: 0.01986041739695841
error: 0.03770711842676848
Updating weights
error: 0.003522810655572943
error: -0.1163774346470

error: 0.05578259736940909
error: 0.3209836022897446
error: 0.3243395686232126
error: 0.278501429081761
error: -0.08192924129447973
error: -0.08146828011142655
error: -0.31425791547220283
error: 0.08344519648694428
error: 0.08305352382003582
error: 0.17113907527347755
Updating weights
error: -0.21210753103883007
error: 0.020037520278944638
error: 0.26317287354179464
error: -0.0057396760693045066
error: -0.006476124253552429
error: 0.042931565752518185
error: 0.022010497029791326
error: 0.021907966793250064
error: 0.04740242246504282
error: 0.2428380170758408
error: 0.24538097329880587
error: 0.1999642899202706
error: -0.0623295637070394
error: -0.0619803154476803
error: -0.29295092166194003
error: 0.06288361449753833
error: 0.06258679616702964
error: 0.14937911412681582
Updating weights
error: 0.12386445078834729
error: 0.061427485682129294
error: -0.24505823669676496
error: 0.0023074989014831918
error: -0.0043217327233307816
error: -0.04757933489180492
error: -0.008242256223198395
err

error: -0.41608430626442555
error: -1.2186224314731827e-05
error: 0.01950246576306697
error: 0.16456006137876428
Updating weights
error: 0.002896016125808487
error: -0.09199420920616197
error: 0.10519216397378033
error: 1.3490456300602599e-06
error: 0.014193584493140485
error: 0.023353988409755957
error: -5.396410614410126e-06
error: 0.001718275674062847
error: 0.006360941520906705
error: -5.786112889022441e-05
error: -0.05061803291409462
error: -0.060738001414829663
error: 1.611566442184477e-05
error: -0.007921889611401327
error: -0.05486302766995319
error: -1.57916926102552e-05
error: 0.005627303163785903
error: 0.02285660727547812
Updating weights
error: 0.0061717225643239376
error: -0.09846624710023466
error: 0.05312433225593266
error: 6.106634441274793e-06
error: 0.01613036320289513
error: 0.018595060712503317
error: -2.4428021952152814e-05
error: 0.0019174091129582477
error: 0.0031629944248000572
error: -0.0002619200798947843
error: -0.05779340929016047
error: -0.0605324875849432

error: 0.03795552086411556
error: 0.05037097349875876
Updating weights
error: 0.006922865349190913
error: -0.08793427018158173
error: 0.03975750003133294
error: 9.57021782766885e-06
error: 0.01318560222802712
error: 0.014534666558624327
error: -3.02526607875838e-05
error: 0.0016078899873109094
error: 0.002292864465859126
error: -0.00033200940792895236
error: -0.04724275965190522
error: -0.048919943423318006
error: 0.00010002650884906685
error: -0.00791516561502167
error: -0.015384081570955981
error: -9.380118967747821e-05
error: 0.0052230129819106266
error: 0.007944844038924058
Updating weights
error: 0.0031062720546360936
error: -0.09587710504978997
error: 0.10732364371807736
error: 1.934039910419176e-06
error: 0.015512549879102685
error: 0.024647069235921755
error: -6.114284264313744e-06
error: 0.001908226116392792
error: 0.00654416453043487
error: -6.71002784385332e-05
error: -0.055366177089054044
error: -0.06673238843361061
error: 2.0215541737760895e-05
error: -0.009442162991817282

error: -0.5465397921103716
error: 0.1583998884377391
error: 0.15773076432471272
error: 0.29539721810397324
Updating weights
error: 0.1756885153685343
error: 0.038601434186456736
error: -0.23089544771200787
error: 0.0053832178375978245
error: 0.002728724699149637
error: -0.03226654820262771
error: -0.01657956955596721
error: -0.01687568517187226
error: -0.03466135159605653
error: -0.17889945280787065
error: -0.16923510496263067
error: -0.11982422369285844
error: 0.05760076876174831
error: 0.059400173046906445
error: 0.267873040554353
error: -0.05283083531525328
error: -0.05389195213040778
error: -0.12933749556800336
Updating weights
error: 0.3626071493687328
error: 0.028152954960383607
error: -0.33104300343545223
error: 0.017120780538157934
error: 0.015693158358122936
error: -0.04773682650570013
error: -0.05514824109297636
error: -0.05530773915442469
error: -0.08786553773348552
error: -0.5894080554804139
error: -0.5842116633662351
error: -0.496069702198278
error: 0.18959576482646753
err